In [444]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import selenium
import pandas as pd
import html5lib
import math


In [519]:
dict_of_schools={'Princeton University':{'Number of professors':326 , 'School ID':780},
                 'Harvard University':{'Number of professors': 567, 'School ID':399},
                 'Columbia University':{'Number of professors': 893, 'School ID':278},
                 'Massachusetts Institute of Technology':{'Number of professors':346 , 'School ID':580},
                 'Yale University':{'Number of professors':464 , 'School ID':1222},
                 'Stanford University':{'Number of professors':611 , 'School ID':953},
                 'University of Chicago':{'Number of professors':507 , 'School ID':1085},
                 'University of Pennsylvania':{'Number of professors':523 , 'School ID':169},
                 'California Institute of Technology':{'Number of professors':93 , 'School ID':148},
                 'Johns Hopkins University':{'Number of professors':830 , 'School ID':464},
                 
                 'Wichita State University':{'Number of professors':1713 , 'School ID':1197},
                 'Western Kentucky University':{'Number of professors':1669 , 'School ID':1176},
                 'University of Charleston':{'Number of professors':94 , 'School ID':1084},
                 'Regent University':{'Number of professors':485 , 'School ID':4375},
                 'University of Hawaii at Hilo ':{'Number of professors':435 , 'School ID':1105},
                 'Cleveland State University ':{'Number of professors':1862 , 'School ID':244},
                 'Husson University':{'Number of professors':221 , 'School ID':426},
                 'Palm Beach Atlantic University':{'Number of professors':357 , 'School ID':753},
                 'Texas Womans University':{'Number of professors':1311 , 'School ID':1014},
                 'University of Texas at Tyler':{'Number of professors':717 , 'School ID':4171},
                 
                }
school_to_run='Stanford University'
number_of_prof=dict_of_schools[school_to_run]['Number of professors']
schoolID=dict_of_schools[school_to_run]['School ID']

The keys for each element in list_of_professor_dicts are:,
'tDept' = Department (eg. science, spanish, law ect),
'tSid' = The school id (eg. 1222 for all Yale professors),
'institution_name' = Name of university,
'tFname' = First name,
'tMiddlename' = Middle name,
'tLname' = Last name,
'tid' = Teacher id,
'tNumRatings' = Number of ratings,
'rating_class' = How is their score (eg. good, average, poor, zero),
'contentType' = Have only seen this to return 'TEACHER',
'categoryType'= Have only seen this to return 'PROFESSOR',
'overall_rating' = Average rating (float)

In [497]:
def get_all(school_id, number_of_professors): #BRUG DEN HER OG KUN DEN HER
    number_of_pages=math.ceil(number_of_professors/20)
    list_of_dfs=[]
    for j in range(1,number_of_pages+1):
        url=f"https://www.ratemyprofessors.com/filter/professor/?&page={j}&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)
        response = requests.get(url)
        list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
        list_of_professor_dicts=list_of_dicts['professors'] 
        list_of_dfs.append(pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(len(list_of_professor_dicts))],
                  ignore_index=True))
    df=pd.concat([list_of_dfs[i] for i in range(len(list_of_dfs))], axis= 0).reset_index(drop=True)
    list_of_teacher_ids=list(df.tid)
    list_of_comments_for_each_teacher=[]
    list_of_tags_for_each_teacher=[]
    for i in range(len(list_of_teacher_ids)):
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'
        my_response = requests.get(my_url)
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_text = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of comments unparsed
        my_tags = soup.findAll('span', {'class':'Tag-bs9vf4-0 hHOVKF'})#list of comments unparsed
        finished_comments=[i.text for i in my_text]
        finished_tags=[i.text for i in my_tags]
        list_of_comments_for_each_teacher.append(finished_comments)
        list_of_tags_for_each_teacher.append(finished_tags)
    df['Comments']=list_of_comments_for_each_teacher
    df['Tags']=list_of_tags_for_each_teacher
    df.to_csv(f'{school_id}.csv')
    
    ##Second dataframe
    list_of_teacher_ids=list(df['tid'])
    list_of_teacher_ids=[int(x) for x in list_of_teacher_ids]
    list_of_comments_and_ids=[]
    for i in range(len(list_of_teacher_ids)):
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'
        my_response = requests.get(my_url, headers={'user-agent': 'For an exam at the university of copenhagen for the course ISDS'})
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_comments = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of tags unparsed
        my_date = soup.findAll('div', {'class':'TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-3 BlaCV'})#list of dates unparsed
        my_quality = soup.findAll('div', {'class':'CardNumRating__StyledCardNumRating-sc-17t4b9u-0 eWZmyX'})#list of dates unparsed
        finished_quality=[i.text for i in my_quality]
        finished_comments=[i.text for i in my_comments]
        finished_dates=[i.text for i in my_date]
        if len(finished_comments)!=0:
            list_of_comments_and_ids.append([finished_comments, [k[-3:] for k in finished_quality[0::2]],[k[-3:] for k in finished_quality[1::2]], finished_dates[0::2],[list_of_teacher_ids[i] for j in range(len(finished_comments))]])
    hej=pd.concat([pd.DataFrame(list_of_comments_and_ids[i]).T for i in range(len(list_of_comments_and_ids))])
    hej.columns=['Comments', 'Quality','Difficulty','Dates', 'TeacherID']
    hej=hej.reset_index(drop=True)
    hej.to_csv(f'{school_id}comments.csv')
    return 


In [520]:
get_all(schoolID, number_of_prof)

In [509]:
hej1=pd.read_csv('399.csv')
hej2=pd.read_csv('399comments.csv')
hej2

,Unnamed: 0,Comments,Quality,Difficulty,Dates,TeacherID
0,0,VERY HELPFUL AND WHATEVER HE SAYS THAT'S WHAT ...,4.5,1.0,"May 10th, 2012",1127241
1,1,NaN,5.0,5.0,"Mar 31st, 2010",1127241
2,2,Very solid class from a master of the subject.,5.0,2.0,"Oct 17th, 2014",1532353
3,3,His class was very unique. I really enjoyed be...,5.0,1.0,"May 15th, 2014",1532353
4,4,"Intelligent, helpful, accessible and funny Fre...",5.0,2.0,"Apr 20th, 2011",1532353
...,...,...,...,...,...,...
1699,1699,No Comments,5.0,3.0,"May 17th, 2006",725853
1700,1700,He is wonderful!!,5.0,3.0,"Nov 30th, 2014",766532
1701,1701,Awesome teachers,5.0,5.0,"Jun 30th, 2010",766532
1702,1702,"If this class doesn't change your life, nothin...",5.0,5.0,"Nov 29th, 2005",766532
